Imports

In [2]:
import math
import time
import sys
import ast
import openai
import tdmclient.notebook
import tdmclient.tools
from threading import Thread, Lock
from object_detection import detect_objects, detection
from copy import deepcopy

Connect Robot, Thymio Suite must be open and thymio must be connected to pc!

In [3]:
await tdmclient.notebook.start()

## Code that will run on the Thymio

In [4]:
%%run_python

state = 0


@onevent
def move(distance):
    global motor_left_target, motor_right_target, timer_period
    if distance > 0:
        timer_period[0] = 160 * distance
        motor_left_target = 200
        motor_right_target = 200
    if distance < 0:
        timer_period[0] = 160 * -distance
        motor_left_target = -200
        motor_right_target = -200
    if distance == 0:
        timer_period[0] = 100
    emit("motion", 1)

@onevent
def rotate(angle):
    global motor_left_target, motor_right_target, timer_period
    if angle > 0:
        timer_period[0] = angle * 26
        motor_left_target = -100
        motor_right_target = 100
    if angle < 0:
        timer_period[0] = -angle * 26
        motor_left_target = 100
        motor_right_target = -100
    if angle == 0:
        timer_period[0] = 100
    emit("motion", 1)

@onevent
def prox():
    global motor_left_target, motor_right_target, prox_horizontal, state
    if state == 0:
        for i in range(5):
            if prox_horizontal[i] > 2000:
                state = 1
                timer_period[0] = 1000
                motor_left_target = 0
                motor_right_target = 0
                emit("problem", 0)
    if state == 3:
        motor_right_target = (prox_horizontal[0] + prox_horizontal[1]) // 20
        motor_left_target = (prox_horizontal[3] + prox_horizontal[4]) // 20


@onevent
def turn(right_speed, left_speed, timer):
    global motor_left_target, motor_right_target, timer_period
    motor_left_target = left_speed
    motor_right_target = right_speed
    if timer == 0:
        timer_period[0] = 100
    else:
        timer_period[0] = timer
    emit("motion", 1)


@onevent
def push(timer):
    global state
    state = 3
    timer_period[0] = timer

@onevent
def stoppush(_):
    global state
    state = 0
    timer_period[0] = 5

@onevent
def timer0():
    global motor_left_target, motor_right_target
    if state == 0:
        timer_period[0] = 0
        motor_left_target = 0
        motor_right_target = 0
        emit("done", 0)
    if state == 1:
        state = 0
    
@onevent
def tracking():
    state = 3 
    
    
@onevent
def senddone():
    emit("done",0)


## Code to run on computer

Controllers and logic auxiliar to the main loop

In [5]:
def detectUniqueObject(detection, mutex_detection, shared_obj_arr: list[str], mutex_arr):
    
    while True:
         
        mutex_detection.acquire()
        obj_detect = detection[0]
        mutex_detection.release()
        
        if obj_detect is not None:
            for cat in obj_detect.categories:
                mutex_arr.acquire()
                if cat[0] not in shared_obj_arr:
                    shared_obj_arr.append(cat[0])
                mutex_arr.release()
         
        time.sleep(0.2)

def check_for_object(detection, mutex, category):
    
    for _ in range(20):
        mutex.acquire()
        obj = detection[0]
        mutex.release()
        if obj is not None:
            for cat in obj.categories:
                if cat[0] == category:
                    return True
        time.sleep(0.1)            
    
    
    return False


class PID:
    
    def __init__(self, kp=1, ki=0, kd=0) -> None:
        self.kp = kp
        self.ki = ki
        self.kd = kd
        
        self.integral = 0
        self.last_value = 0
        self.last_error = 0
        self.last_update = time.time_ns()
        
        self.max_integral = 100
        
    def compute(self, setpoint, value):
        now = time.time()
        dt = now - self.last_update
        
        # compute error
        error = setpoint - value
        
        # compute integral and derivative terms
        self.integral += error * dt
        if self.integral > self.max_integral:
            self.integral = self.max_integral
            
        derivative = (error - self.last_error) / dt
        
        output = self.kp * error + self.ki * self.integral + self.kd * derivative
        
        self.last_error = error
        self.last_update = now
        
        return output

def calculate_turn(radius, angle):
    wheel_distance = 10.1
    velocity = 5.8
    
    radian_angle = math.radians(angle)  # Convert angle from degrees to radians
    arc_length = radius * abs(radian_angle)  # Calculate the arc length the robot needs to travel

    # Calculate the target velocities for each wheel
    if angle == 0:
        motor_left_target = motor_right_target = velocity
    else:
        motor_right_target = velocity * (1 + wheel_distance / (2 * radius)) if angle > 0 else velocity * (1 - wheel_distance / (2 * radius))
        motor_left_target = velocity * (1 - wheel_distance / (2 * radius)) if angle > 0 else velocity * (1 + wheel_distance / (2 * radius))

    return to_int(motor_left_target * 100/3.2), to_int(motor_right_target * 100/3.2), to_int((arc_length / velocity) * 1000)


def get_expected_time(event_name, args):
    match event_name:
        case "move":
            return 0.16 * args[0]
        case "rotate":
            return 0.026 * args[0]

def clamp(value, min_value, max_value):
    return max(min(value, max_value), min_value)

# Event handling exceptions

class Done(Exception):
    pass

class Problem(Exception):
    pass

class EventTimeout(Exception):
    pass

class ObjectNotFound(Exception):
    pass

# In case the robot fails to receive an event (happens alot :( ) raise this expection
def timeout_handler(sig, NULL):
    raise EventTimeout

def parse_input(task_str):
    actions = ast.literal_eval(task_str)
    return actions

def to_int(str):
    return int(round(float(str)))
    

Unchecked Raise(exc=Name(id='EventTimeout', ctx=Load()))


ChatGPT Integration

In [10]:
class ChatApp:
    
    OBJECT_NOT_FOUND = """The object you were trying to follow was not in front of you.
    Please give the robot new movement instructions in the form of a list. Include the previous
    follow instruction at the end please.
    """
    
    PROBLEM = """The robot encountered an obstacle in front of it.
    Please instruct the robot how to go move out of it's way, so that it can resume its previous actions.
    """
    
    
    messages = []
    
    
    def __init__(self):
        with open("apikey", "r") as file:
            key = file.read()
        openai.api_key = key
        
    def begin(self):
        
        message = [
        {"role": "user", "content": 
         """Imagine you are a two wheeled robot with a wheelabase of 10 cm that can move in a XY plane,
          you move through a differential drive system. The main functions you can use
          to move arround are:
          move(x: int): Moves the robot linearly either x cm to the front or x cm backwards.
          rotate(angle: int): Rotates the robot by a number of degrees without changing the XY position. Positive angles are to the right.
          turn(radius where radius > 5, angle): Performs a turn with the given radius and with the correspoding arc to the angle. Radius cannot be negative and must greater than 5. Positive angles are right turns
          follow(object): Follows the object if it is in front of the robot.
          
          You can give a list of actions for the robot to perform, for example:
          [("turn",10,90),("rotate",45),("move",-10)]

          Only respond in these lists

          Here is your task:

          """  + input() + """\nONLY RESPOND IN LIST OF TASKS""",
          }
        ]
        
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages= message
        )
        
        self.messages.append(message[0])
        self.messages.append({"role": "assistant", "content": response["choices"][0]["message"].content})
        
        print(response["choices"][0]["message"].content)
        
        return response["choices"][0]["message"].content
    
    def ask_where(self, list_objects):
        
        
    def chat(self, message):
        self.messages.append({"role": "user", "content": message})
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=self.messages
        )
        self.messages.append({"role": "assistant", "content": response["choices"][0]["message"].content})
        return response["choices"][0]["message"].content

Robot main logic

In [7]:
import signal

@tdmclient.notebook.sync_var
def update_speed(left, right):
    set_var(motor_left_target = left, motor_right_target = right)

@tdmclient.notebook.sync_var
def get_current_speed():
    return motor_left_target, motor_right_target

@tdmclient.notebook.sync_var
def get_prox():
    return prox_horizontal

@tdmclient.notebook.sync_var
def set_timer(time, i=0):
    timer_period = time * 1000
    set_var(timer_period = timer_period)

DETECTED_OBJECTS = [None]
MUTEX = Lock()

UNIQUE_OBJECTS = [None]
MUTEX_ARR = Lock()

RobotGPT = ChatApp()

@tdmclient.notebook.sync_var
def object_tracking(object_category: str):

    # Max turn speed
    max_speed = 50
    min_speed = -50

    # PID Controller
    pid = PID(0.1,0,0.1)
    
    # Event timeout 
    timer = 0
    
    # Going towards an object flag
    towards = False

    while True:
        MUTEX.acquire()
        detect_ref = DETECTED_OBJECTS[0]
        if detect_ref is not None:
            detect = deepcopy(detect_ref)
        else:
            detect = None
        DETECTED_OBJECTS[0] = None
        MUTEX.release()
        
        left_speed, right_speed = get_current_speed()                     
        
        if detect is not None:
            # Get the current object position (x, y) from the camera
            object_x, x_left, x_right = detect.get_position_of_object(object_category)
            if object_x is not None:
                x_size = x_right - x_left
                obj_size = x_size / 480

                # Apply proportional control
                speed_adjustment = pid.compute((480//2 - 65), object_x)
            
                if abs(speed_adjustment) < 5 + 10 * obj_size and not towards:
                    towards = True
                    update_speed(200,200)
                    time.sleep(0.075)
                elif towards:
                    if abs(speed_adjustment) > 5 + 10 * obj_size:
                        towards = False
                        update_speed(0,0)
                        time.sleep(0.075)
                    else:
                        time.sleep(0.075)
                else:                
                    # Update the wheel speeds based on the control algorithm
                    new_left_speed = to_int(clamp(left_speed - speed_adjustment, min_speed, max_speed))
                    new_right_speed = to_int(clamp(right_speed + speed_adjustment, min_speed, max_speed))

                    update_speed(new_left_speed, new_right_speed)
                    
                    time.sleep(0.075)
                timer = 0
            else:
                timer += 1
                if timer == 20:
                    towards = False
                    timer = 0
                    update_speed(0,0)
                    break
                time.sleep(0.075)
        else:
            timer += 1
            if timer == 20:
                towards = False
                timer = 0
                update_speed(0,0)
                break
            time.sleep(0.075)
        # Sleep for a while to let the robot move and adjust its position
    return "Done"
        

def pop_and_send(event_list, past_events) -> bool:
    if len(event_list) > 0:
        event = event_list.pop(0)
        past_events.append(event)
        event_name = event[0]
        match event_name:
            case "turn":
                right_speed, left_speed, timer = calculate_turn(event[1], event[2])
                expected_time = timer / 1000
                print(right_speed, left_speed, timer)
                send_event("turn", right_speed, left_speed, timer)
                print("Sent event")
            case "move":
                expected_time = get_expected_time(event[0], event[1:])
                send_event(event[0], to_int(event[1]))
                print("Sent event")
            case "rotate":
                expected_time = get_expected_time(event[0], event[1:])
                send_event(event[0], to_int(event[1]))
                print("Sent event")
            case "follow":
                expected_time = 0
                object_name = event[1]
                if check_for_object(DETECTED_OBJECTS, MUTEX, object_name):
                    send_event("tracking")
                    object_tracking(object_name)
                    send_event("senddone")
                else:
                    raise ObjectNotFound
            case "push":
                pass
            case _:
                expected_time = get_expected_time(event[0], event[1:])
                send_event(event[0], *event[1:])
                print("Sent event")
        # Set an alarm with the aproximate time the robot will take to complete
        # the action plus a 1 second delay
        if expected_time != 0:
            signal.alarm(int(expected_time + 2))
        return True
    else:
        return False

def execute_actions(actions): 
    # Define handler for the alarm
    signal.signal(signal.SIGALRM, timeout_handler)
    event_list = actions.copy()
    past_events = []

    def on_event_data(node, event_name):
        #print(f"Receiving event data for event {event_name}")
        data = get_event_data(event_name)
        #print(data)
        if event_name == "done":
            clear_event_data(event_name)
            success = pop_and_send(event_list, past_events)
            if not success:
                signal.alarm(0)  # Cancel the alarm
                raise Done
        elif event_name == "problem":
            signal.alarm(0)
            clear_event_data(event_name)
            raise Problem
        elif event_name == "motion":
            clear_event_data(event_name)
        else:
            raise Exception("Unknown event")
    
    try:
        pop_and_send(event_list, past_events) 
        tdmclient.notebook.process_events(all_nodes=False, on_event_data=on_event_data)
    except Done:
        print("Robot task completed successfully")
        return "done", None
    except Problem:
        print("Robot encounted a problem")
        return "problem", event_list
    except EventTimeout:
        print("Missed event, resending")
        event_list.insert(0, past_events[-1])
        return execute_actions(event_list)
    except ObjectNotFound:
        print("Couldn't find object, asking ChatGPT for directions")
        response = RobotGPT.chat(RobotGPT.OBJECT_NOT_FOUND)
        print(response)
        new_actions = parse_input(response)
        return execute_actions(new_actions)


@tdmclient.notebook.sync_var
def main(response=None):
    
    # Initiate thread that will receive tensorflow detections
    tensor_flow_client = Thread(target=detect_objects, args=(DETECTED_OBJECTS, MUTEX))
    unique_object_detection = Thread(target=detectUniqueObject, args=(DETECTED_OBJECTS, MUTEX, UNIQUE_OBJECTS, MUTEX_ARR))
    try:
        tensor_flow_client.start()
        unique_object_detection.start()
        quit = False
        if response is None:
            tasks = [parse_input(input("Please input a list of actions for the robot to perform"))]
        else:
            chatgpt_response = RobotGPT.begin()
            tasks = [parse_input(chatgpt_response)]
        while not quit:
            if len(tasks) > 0:
                response, left_over_events = execute_actions(tasks.pop())
                match response:
                    case "done":
                        quit = True
                    case "problem":
                        if left_over_events is not None:
                            tasks.append(left_over_events)
                        tasks.append(parse_input(input("Robot encoutered a problem (*Insert short description*).\nPlease provice extra guidance.")))
            else:
                break
    except KeyboardInterrupt:
        sys.exit()

Unchecked Raise(exc=Name(id='ObjectNotFound', ctx=Load()))
Unchecked MatchAs()


Stop the robot

In [8]:
update_speed(0,0)

Initiate robot

In [9]:
main(1)

trying to connect
sucessfully connected
[("turn",6,90),("follow","bottle")]
29 334 1625
Sent event
Missed event, resending
29 334 1625
Sent event
Missed event, resending
Couldn't find object, asking ChatGPT for directions
[("move",5),("turn",10,-90),("move",5),("follow","bottle")]
Sent event
Missed event, resending
Sent event
Missed event, resending
Sent event
273 90 2708
Sent event
Sent event
Missed event, resending
Couldn't find object, asking ChatGPT for directions
[("turn",6,90),("move",10),("turn",6,-90),("move",10),("follow","bottle")]
29 334 1625
Sent event
Missed event, resending
29 334 1625
Sent event
Missed event, resending
29 334 1625
Sent event
Missed event, resending
29 334 1625
Sent event
invalid literal for int() with base 10: ''
Connection error
trying to connect
sucessfully connected
Sent event
334 29 1625
Sent event
Sent event
Missed event, resending
Couldn't find object, asking ChatGPT for directions
[("move",5),("turn",20,90),("move",5),("turn",20,-90),("move",5),("

TypeError: cannot unpack non-iterable NoneType object